In [2]:
# Get tensorboard running in the background
LOG_DIR = '/tmp/log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

# Download and unzip ngrok
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

# Launch ngrok background process
get_ipython().system_raw('./ngrok http 6006 &')

# Retrive public url
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


Redirecting output to ‘wget-log.1’.
Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
http://79df091e.ngrok.io


In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will proot for authorization
drive.mount('/content/drive')

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

# NAME = "Cats-vs-dog-cnn-64x2-{}".format(int(time.time()))


X = pickle.load(open("/content/drive/My Drive/Colab Notebooks/cnn_own_data/X.pickle", "rb"))
y = pickle.load(open("/content/drive/My Drive/Colab Notebooks/cnn_own_data/y.pickle", "rb"))

X = X/255.0

dense_layers = [0] # [0, 1, 2]
layer_sizes = [64] # [32, 64, 128]
conv_layers = [3] # [1, 2, 3]

for dense_layer in dense_layers:
  for layer_size in layer_sizes:
    for conv_layer in conv_layers:
      NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
      print(NAME)
      tensorboard = TensorBoard(log_dir='/tmp/log/{}'.format(NAME)) # TensorBoard(log_dir='logs/{}'.format(NAME))
      
      model = Sequential()
      
      model.add(Conv2D(layer_size, (3,3), input_shape=X.shape[1:]))
      model.add(Activation("relu"))
      model.add(MaxPooling2D(pool_size=(2,2)))
      
      for l in range(conv_layer-1):
        model.add(Conv2D(layer_size, (3,3)))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2)))
      
      model.add(Flatten()) # this converts our 3D feature maps to 1D feature vectors
      for l in range(dense_layer):
        model.add(Dense(layer_size))
        model.add(Activation('relu'))

      model.add(Dense(1))
      model.add(Activation("sigmoid"))

      model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

      model.fit(X, y, batch_size=32, epochs=32, validation_split=0.1, callbacks=[tensorboard])

model.save('/content/drive/My Drive/Colab Notebooks/cnn_own_data/64x3-CNN.model') # model.save('64x3-CNN.model')
'''
Best versions of the conv network (used tensorboard to analyze data)
3 conv, 64 nodes per layer, 0 dense
3 conv, 128 nodes per layer, 0 dense
3 conv, 32 nodes per layer, 0 dense
3 conv, 32 nodes per layer, 2 dense
3 conv, 32 nodes per layer, 1 dense
2 conv, 32 nodes per layer, 0 dense
2 conv, 64 nodes per layer, 0 dense
3 conv, 128 nodes per layer, 1 dense
2 conv, 128 nodes per layer, 0 dense
2 conv, 32 nodes per layer, 1 dense
'''

Mounted at /content/drive
3-conv-64-nodes-0-dense-1539049257
Train on 22451 samples, validate on 2495 samples
Epoch 1/32
22451/22451 [==============================] - 11s 511us/step - loss: 0.6373 - acc: 0.6267 - val_loss: 0.5814 - val_acc: 0.6978
Epoch 2/32
22451/22451 [==============================] - 11s 483us/step - loss: 0.5369 - acc: 0.7332 - val_loss: 0.4968 - val_acc: 0.7635
Epoch 3/32
22451/22451 [==============================] - 11s 491us/step - loss: 0.4800 - acc: 0.7718 - val_loss: 0.4846 - val_acc: 0.7663
Epoch 4/32
22451/22451 [==============================] - 11s 480us/step - loss: 0.4382 - acc: 0.7949 - val_loss: 0.4409 - val_acc: 0.8000
Epoch 5/32
22451/22451 [==============================] - 11s 477us/step - loss: 0.4036 - acc: 0.8161 - val_loss: 0.4303 - val_acc: 0.8076
Epoch 6/32
22451/22451 [==============================] - 11s 488us/step - loss: 0.3718 - acc: 0.8311 - val_loss: 0.4242 - val_acc: 0.8080
Epoch 7/32
22451/22451 [==============================] 